# Implementação do algoritmo de otimização Simulated Anealling

### Rafael Brusiquesi Martins

## Importando bibliotecas

In [1]:
import numpy as np
from copy import deepcopy

## Definindo a classe SimulatedAnealling para aplicar o algoritmo de otimização proposto.

In [2]:
class SimulatedAnealling:
    def __init__(self, dimension, initial_temperature, max_random_radius, low_boundaries=None, high_boundaries=None):
        self.dimension = dimension
        self.initial_temperature = initial_temperature
        self.max_random_radius = max_random_radius

        self.low_boundaries = np.full(self.dimension, -np.inf) if low_boundaries is None else low_boundaries
        self.high_boundaries = np.full(self.dimension, np.inf) if high_boundaries is None else high_boundaries
    
    def new_guess(self, guess):
        new_guess = deepcopy(guess)
        new_guess += self.max_random_radius * np.random.uniform(-1, 1, size=self.dimension)

        return new_guess
    
    def maximize(self, function, initial_guess, n_iterations=10000):
        temperature = self.initial_temperature

        guess = initial_guess
        function_value = function(guess)
        best_guess = deepcopy(guess)
        
        for i in range(n_iterations):
            new_guess = np.clip(self.new_guess(guess), self.low_boundaries, self.high_boundaries)
            new_function_value = function(new_guess)

            if new_function_value > function_value:
                guess = new_guess
                function_value = new_function_value
            
            else:
                criterion = np.exp(-(function_value - new_function_value)/temperature)
                
                if np.random.uniform() < criterion:
                    guess = new_guess
                    function_value = new_function_value
            
            if function_value > function(best_guess):
                best_guess = deepcopy(guess)

            temperature = self.initial_temperature / (i + 1)
        
        return best_guess
    
    def minimize(self, function, initial_guess, n_iterations=10000):
        negative_function = lambda x: -function(x)

        return self.maximize(negative_function, initial_guess, n_iterations)
    
    def multiple_restart_optimization(self, function, initial_guess, n_restarts, n_iterations=10000, operation='maximize'):
        if operation == 'maximize':
            ans = self.maximize(function, initial_guess, n_iterations=n_iterations)
            for i in range(n_restarts):
                ans = self.maximize(function, initial_guess=ans, n_iterations=n_iterations)

        elif operation == 'minimize':
            ans = self.minimize(function, initial_guess, n_iterations=n_iterations)
            for i in range(n_restarts):
                ans = self.minimize(function, initial_guess=ans, n_iterations=n_iterations)

        else:
            raise Exception('incorrect operation keyword')
        
        return ans


## Definindo uma função de teste para o algoritmo.

In [3]:
def f(x):
    return np.sin(x[0]) + np.cos(x[1]) - 0.002 * x[0]**2 - 0.002 * x[1]**2

Construindo um objeto do tipo SimulatedAnealling e definindo parâmetros de temperatura, dimensão e raio máximo de deslocamento aleatório por etapa.
Aplicando a otimização pelo número de iterações desejadas para um chute inicial fornecido.

In [4]:
SA = SimulatedAnealling(
    dimension=2,
    initial_temperature=10000,
    max_random_radius=1
    )

ans = SA.maximize(f, initial_guess=np.ones(shape=SA.dimension), n_iterations=100000)
print(ans, f(ans))

[ 1.47424216 -0.10639476] 1.9853182619351644


Com este teste pode-se perceber que o algoritmo se apresenta bem robusto para encontrar soluções no espaço bidimensional, visto que a função objetivo utilizada possui característica de multiplicidade de ótimos locais e valores bem próximos entre eles, mas ainda assim, o algoritmo conseguiu se aproximar ao ótimo global. Uma opção para atingir o ponto de ótimo global seria aplicar um algoritmo de gradiente descendente/ascendente a partir do resultado obtido pelo recozimento simulado.